# Incialização

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.subplots import make_subplots
from IPython.display import display_html
import pm4py

ModuleNotFoundError: ignored

# Funções

In [ ]:
def outliers(df, limite = 3, coluna='Duração (s)'):
  return df[df[coluna] > limite]

def cria_df(worksheet):
  df = pd.DataFrame.from_records(
      worksheet.get_all_values()[1:], 
      columns= worksheet.get_all_values()[0] 
      #columns= ['atividade', 'projeto', 'inicio', 'fim', 'duracao']
      )
  df['Duração (s)'] = round(df['Duração (s)'].astype('int')/3600, 3)
  df['user'] = worksheet.title.replace(' Auxiliar','')
  try:
    df.drop('', axis=1,inplace=True)
  except:
    pass
  return df

def cria_df_2(worksheet):
  filtro = lambda row: row[2]!=''
  values = list(map(lambda r: r[:5],filter(filtro, worksheet.get_all_values())))[:-2]
  
  df = pd.DataFrame.from_records(
      values[1:], 
      columns= ['Equipe',	'Descrição da atividade',	'Início',	'Fim',	'Duração (s)'] 
      #columns= ['atividade', 'projeto', 'inicio', 'fim', 'duracao']
      )
  df['Duração (s)'] = round(df['Duração (s)'].astype('int')/3600, 3)
  df['user'] = worksheet.title.replace(' (Auxiliar)','')
  return df

def cria_df_3(worksheet):
  values = worksheet.get_all_values()
  df = pd.DataFrame(values[1:], columns=values[0])
  df['quantity'] = round(df['quantity'].astype('int')/3600, 3)
  return df


def month_column(df):
  mes = lambda x: int(x[3:5])
  df['mes'] = list(map(mes, df['Início']))
  return df

def year_column(df):
  ano = lambda x: int(x[6:10])
  df['ano'] = list(map(ano, df['Início']))
  return df

def new_columns(df):
  df = year_column(df)
  df = month_column(df)
  return df

def filter_before(df, type, lower_or_equal_than):
  return df[df[type] <= lower_or_equal_than]

def filter_after(df, type, since):
  return df[df[type] >= since]

def contar_vazios(df, coluna):
  return df.set_index(coluna).isin(['']).sum(level=0)

def taxa_vazios(df, coluna):
  return round(df.set_index(coluna).isin(['']).sum(level=0) / df.groupby('user').count(),3)

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

# Análise geral

Abre todos os arquivos

In [ ]:
coh_20_2 = gc.open('2020.2 Controle de Horas Mensais - Inderios')
coh_21_1 = gc.open('2021.1 Controle de Horas Mensais - Inderios')
coh_21_2 = gc.open('2021.2 Controle de Horas Mensais - Inderios')

Concatena todos os dados de cada semestre em somente uma tabela

In [ ]:
nomes_1 = coh_20_2.worksheets()
df_20_2 = pd.DataFrame()
#Cria dataframe unificado com todos os registros de todo mundo
for aba in nomes_1:
  if 'Auxiliar' in aba.title:
    df_20_2 = df_20_2.append(cria_df(aba), ignore_index=True)

nomes_2 = coh_21_1.worksheets()
df_21_1 = pd.DataFrame()
#Cria dataframe unificado com todos os registros de todo mundo
for aba in nomes_2:
  if 'Auxiliar' in aba.title and aba.title != 'Base (Auxiliar)':
    df_21_1 = df_21_1.append(cria_df_2(aba), ignore_index=True)

nomes_3 = coh_21_2.worksheets()[7:-4]
df_21_2 = pd.DataFrame()
#Cria dataframe unificado com todos os registros de todo mundo
for aba in nomes_3:
  df_21_2 = df_21_2.append(cria_df_3(aba), ignore_index=True)

KeyError: ignored

## Missing values por semestre

### 2020.2

In [ ]:
abs_20_2 = contar_vazios(df_20_2,'user')
taxa_20_2 = taxa_vazios(df_20_2,'user')

print('- Total de valores faltantes:\n{}'.format(abs_20_2.sum()))
print('\n- Taxa por coluna:\n{}'.format(abs_20_2.sum() / df_20_2.shape[0]))

In [ ]:
display_side_by_side(contar_vazios(df_20_2,'user'),taxa_vazios(df_20_2,'user'))

,Descrição da atividade,ID Equipe,Equipe,Início,Fim,Duração (s)
user,,,,,,
André Sousa,0,0,0,0,5,0
André Tavares,60,154,0,0,2,0
Vitor Paiva,0,0,0,0,78,0
Júlia,2,2,0,0,4,0
Anna Júlia,0,0,0,0,1,0
Anselmo,4,31,0,0,20,0
Artur,9,47,642,0,3,0
Aruana,0,8,0,0,2,0
Camila,0,0,0,0,2,0


### 2021.1

In [ ]:
abs_21_1 = contar_vazios(df_21_1,'user')
taxa_21_1 = taxa_vazios(df_21_1,'user')

print('- Total de valores faltantes:\n{}'.format(abs_21_1.sum()))
print('\n- Taxa por coluna:\n{}'.format(abs_21_1.sum() / df_21_1.shape[0]))

- Total de valores faltantes:
Equipe                     0
Descrição da atividade    55
Início                     0
Fim                       43
Duração (s)                0
dtype: int64

- Média por coluna:
Equipe                    0.000000
Descrição da atividade    0.100895
Início                    0.000000
Fim                       0.009000
Duração (s)               0.000000
dtype: float64


In [ ]:
display_side_by_side(contar_vazios(df_21_1,'user'),taxa_vazios(df_21_1,'user'))

,Equipe,Descrição da atividade,Início,Fim,Duração (s)
user,,,,,
André Tavares,0,0,0,0,0
Anna Clara,0,0,0,4,0
Anselmo,0,1,0,2,0
Aruana,0,0,0,0,0
Bernardo Tenório,0,43,0,4,0
Caio Hassan,0,1,0,0,0
Elisa Vassoler,0,0,0,4,0
Emily Christine,0,0,0,3,0
Gabriela Luiza,0,0,0,4,0


### 2021.2

In [ ]:
abs_21_2 = contar_vazios(df_21_2,'user')
taxa_21_2 = taxa_vazios(df_21_2,'user')

print('- Total de valores faltantes:\n{}'.format(abs_21_2.sum()))
print('\n- Taxa por coluna:\n{}'.format(abs_21_2.sum() / df_21_2.shape[0]))

- Total de valores faltantes:
description    102
date             0
quantity         0
project         78
dtype: int64

- Taxa por coluna:
description    0.012078
date           0.000000
quantity       0.000000
project        0.009236
dtype: float64


In [ ]:
display_side_by_side(contar_vazios(df_21_2,'user'),taxa_vazios(df_21_2,'user'))

,description,date,quantity,project
user,,,,
Allan Santos,1,0,0,2
Ana Flávia Guedes,18,0,0,20
Andre Tavares,0,0,0,0
Maria Clara Cabral,0,0,0,1
Anna Clara Guedes,3,0,0,6
Bernardo Tenorio,15,0,0,2
Caio Hassan,0,0,0,1
Daniela Reis,13,0,0,14
Elisa Vassoler,0,0,0,0


Por projeto

In [ ]:
df_21_2.drop(['user','project'], 1).isin(['']).groupby([df_21_2.project, df_21_2.user], sort=False).sum().reset_index().groupby('project').sum()

,description,date,quantity
project,,,
,44,0,0
Be Change,0,0,0
Betânia Park,0,0,0
Case,1,0,0
Case de Férias,1,0,0
Cogestão e Trainee,0,0,0
Cogestão e Treinee,0,0,0
Cogestão e trainee,0,0,0
Cogestão/Treinee,0,0,0


# Sanity check 2020.2


-Tirar lucca (só um projeto), artur (com registros do kelvin), Mariana (nem 20% dos registros com código de projeto), Sarah (somente registros de agosto)

In [ ]:
remover_nomes = [
                 'Artur',
                 'Lucca Falleiro',
                 'Mariana',
                 'Sarah'
              ]

mask = df_20_2['user'].isin(remover_nomes)
df_20_2 = df_20_2[~mask]

**Pegar somente entradas maiores que 10 segundos**

In [ ]:
df_20_2 = df_20_2[df_20_2['Duração (s)'] > 0.003]

Cria as colunas ano e mes

In [ ]:
df_20_2 = new_columns(df_20_2)

Somente dados de agosto a dezembro de 2020


In [ ]:
df_20_2 = filter_after(df_20_2,'mes',8)
df_20_2 = filter_before(df_20_2,'mes',12)
df_20_2 = filter_after(df_20_2,'ano',2020)

Corrige as atividades que possuem reunião no nome e possuem mais de 4 horas para ficarem somente com 2 horas.

In [ ]:
# Analisa quais outliers são reuniões
out = outliers(df_20_2, limite=4)

# Coloca os outliers que possuem reunião no nome com duracao de 2 horas
index_reunioes = out[list(map(lambda df_row: 'reuni' in df_row.lower(), out['Descrição da atividade']))].index

df_20_2.loc[index_reunioes, 'Duração (s)'] = 2
out.drop(index_reunioes, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Remove as entradas de quando esqueceram o toggl desligado: são entradas que possuem descrição, projeto e data de início iguais e uma linha com fim (de quando o toggl foi desligado) e uma linha com campo fim vazio (de quando o toggl estava ligado e registrou de madrugada)

In [ ]:
df_len = len(df_20_2)

# As entradas que não possuem fim estão duplicadas
indice_remover = out[(out.duplicated(['Descrição da atividade', 'ID Equipe','Início'], keep=False)) & (out['Fim'] == '')].index
df_20_2.drop(indice_remover, inplace=True)
out.drop(indice_remover,inplace=True)

print('{} linhas removidas'.format(df_len - len(df_20_2)))

14 linhas removidas


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Corrige atividades com duração maior que 4 horas

In [ ]:
# Atividades com duração maior que 4.1 horas ficam com duração média das atividades
media = df_20_2.loc[(df_20_2['Duração (s)'] <= 4) & (df_20_2['Duração (s)'] > 0.1)]['Duração (s)'].mean()

df_20_2.loc[out.index, 'Duração (s)'] = media

Substitui "Indústria de Calçados Giovanna da Mata" por "Indústria"

In [ ]:
df_20_2.loc[df_20_2['Equipe'] == 'Indústria de Calçados Giovanna da Mata', 'Equipe'] = 'Industria'

# Análise 2020.2

## Geral

### Tabela consolidada

In [ ]:
# Cria tabela com carga horária por mês para cada pessoa

num_meses = [8,9,10,11,12]
meses = ['agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

tabela_controle = pd.DataFrame(columns=meses)

for nome in df_20_2['user'].unique():
  dic = {'Nome':nome}

  for num_mes, mes in zip(num_meses,meses):
    dic[mes] = round(df_20_2.loc[(df_20_2['user'] == nome) & (df_20_2['mes'] == num_mes), 'Duração (s)'].sum(), 2)

  tabela_controle = tabela_controle.append(dic, ignore_index=True)

# Coloca o nome como index e remove a coluna Nome
tabela_controle.set_index(tabela_controle['Nome'],inplace=True)
tabela_controle.drop('Nome',axis=1,inplace=True)

tabela_controle

,agosto,setembro,outubro,novembro,dezembro
Nome,,,,,
André Sousa,39.06,44.17,51.99,15.87,7.62
André Tavares,82.10,108.84,141.06,51.41,65.27
Vitor Paiva,65.98,90.91,89.94,40.04,17.61
Júlia,81.77,93.27,100.55,37.33,65.14
Anna Júlia,33.50,54.99,52.17,15.97,6.43
Anselmo,61.29,87.70,72.22,44.54,28.80
Aruana,61.25,70.50,72.98,31.50,10.75
Camila,79.16,67.21,78.37,24.72,43.80
Flávio,70.55,69.10,74.42,30.21,19.87


In [ ]:
tabela_controle.describe()

,agosto,setembro,outubro,novembro,dezembro
count,20.000000,20.000000,20.000000,20.000000,20.000000
mean,69.793500,74.156000,75.190500,30.162000,27.036500
std,28.243107,22.173444,28.134228,14.674941,19.255253
min,32.930000,36.940000,27.970000,10.630000,6.430000
25%,55.205000,60.090000,52.125000,16.540000,11.065000
50%,63.635000,69.800000,73.700000,28.695000,19.030000
75%,81.852500,91.500000,92.592500,39.650000,41.270000
max,152.010000,113.980000,141.060000,62.100000,65.270000


## Projetos

In [ ]:
mask = ['Núcleo','-', 'Eleições', 'Case de férias', 'Diretoria Estratégica',
        'GRH', 'Eleição (GRH)', 'Troca de gestão', 'Suporte', 'DP','DRH']

projetos = ['Remoda', 
            'Industria',
            'Vovó Taninha',
            'Projeto Badu',
            'Vort3x',
            'Cheiro de Bolo']

internas = ['PSE',
            'Mídias',
            'Parcerias',
            'Eventos',
            'PS']

df_projetos = df_20_2.loc[df_20_2['Equipe'].isin(projetos)]
df_equipes = df_20_2.loc[df_20_2['Equipe'].isin(internas)]

### Violin plot projetos a cada mês

In [ ]:
# BARRAS DA MÉDIA DE CADA EQUIPE
sub = make_subplots(rows=5, cols=1,
                    subplot_titles=projetos
                    )

for m in [8,9,10,11,12]:
  fig = go.Figure()
  df_m = df_20_2[df_20_2['mes'] == m]

  for proj in projetos:
    fig.add_trace(go.Violin(y=df_m[df_m['Equipe'] == proj].groupby('user')['Duração (s)'].sum().values,
                        name=proj))
  fig.update_layout(title={'text': 'Mês ' + str(m)},
                    height = 300)
  fig.show()


### Total de horas por projeto

In [ ]:
# fig = make_subplots(rows=2, cols=3, subplot_titles=projetos)
fig = make_subplots(rows=1, cols=6, 
                    #subplot_titles=projetos
                    )

for num, proj in enumerate(projetos):
  # lin = 1
  # if num >= 3:
  #   num = num - 3
  #   lin = 2

  total = df_20_2[df_20_2['Equipe'] == proj].groupby('user').sum()['Duração (s)'].sort_values(ascending=False)
  fig.add_trace(
      go.Bar(name=proj,x=total.index, y=total.values),
      # row=lin, col= num + 1
      row=1, col= num + 1
  )

fig.update_layout(height=600, 
                  width=1500, 
                  title_text="Total de horas por projeto",
                  )
fig.update_yaxes(range=[0,130])

fig.show()

### Gráfico média mensal por projeto

In [ ]:
meses = ['agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

fig = make_subplots(rows=1, cols=6, 
                    subplot_titles=meses
                    )

df_media_mensal = pd.DataFrame(index=projetos)

for num, mes in enumerate(range(8,13)):
  grouped_data =  df_projetos[df_projetos['mes'] == mes].groupby(['Equipe','user']).sum()
  data2plot = grouped_data.groupby(level=0).mean()['Duração (s)']
  fig.add_trace(
     go.Bar(name=mes, x=data2plot.index, y=data2plot.values),
     row=1, col= num + 1
  )
  df_media_mensal[meses[num]] = data=data2plot.values

fig.update_layout(height=600, 
                  width=1500, 
                  title_text="Média mensal por projeto",
                  )

fig.update_yaxes(range=[0,31])

fig.show()

### Tabela média mensal por projeto

In [ ]:
df_media_mensal.round(2)

,agosto,setembro,outubro,novembro,dezembro
Remoda,19.44,28.11,20.55,9.82,2.41
Industria,16.37,23.14,23.30,10.70,3.57
Vovó Taninha,19.38,28.37,30.23,10.77,3.57
Projeto Badu,23.87,20.34,18.61,8.48,3.59
Vort3x,19.45,19.65,21.67,8.45,2.84
Cheiro de Bolo,21.09,21.32,20.98,10.35,4.34


### Média geral por mês

In [ ]:
df_media_mensal.mean().round(2)

agosto      19.93
setembro    23.49
outubro     22.56
novembro     9.76
dezembro     3.39
dtype: float64

In [ ]:
#           MÊS
# PROJETO MÉDIA DESVIOPADRÃO
# nome      15        3

## Equipes internas

### Violin plot equipes internas a cada mês

In [ ]:
# BARRAS DA MÉDIA DE CADA EQUIPE
sub = make_subplots(rows=5, cols=1,subplot_titles=projetos)

for m in [8,9,10,11,12]:
  fig = go.Figure()
  df_m = df_20_2[df_20_2['mes'] == m]

  for proj in internas:
    fig.add_trace(go.Violin(y=df_m[df_m['Equipe'] == proj].groupby('user')['Duração (s)'].sum().values,
                        name=proj))
  fig.update_layout(title={'text': 'Mês ' + str(m)},
                    height = 400
                    )
  fig.show()

### Total de horas por equipe

In [ ]:
# fig = make_subplots(rows=2, cols=3, subplot_titles=projetos)
fig = make_subplots(rows=1, cols=6, 
                    subplot_titles=internas
                    )

for num, proj in enumerate(internas):
  # lin = 1
  # if num >= 3:
  #   num = num - 3
  #   lin = 2

  total = df_20_2[df_20_2['Equipe'] == proj].groupby('user').sum()['Duração (s)'].sort_values(ascending=False)
  fig.add_trace(
      go.Bar(name=proj,x=total.index, y=total.values),
      # row=lin, col= num + 1
      row=1, col= num + 1
  )

fig.update_layout(height=600, 
                  width=1500, 
                  title_text="Total de horas por projeto",
                  )
fig.update_yaxes(range=[0,170])

fig.show()

### Gráfico média mensal por equipe

In [ ]:
meses = ['agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

fig = make_subplots(rows=1, cols=6, 
                    subplot_titles=meses
                    )

df_media_mensal = pd.DataFrame(index=internas)

for num, mes in enumerate(range(8,13)):
  grouped_data =  df_equipes[df_equipes['mes'] == mes].groupby(['Equipe','user']).sum()
  data2plot = grouped_data.groupby(level=0).mean()['Duração (s)']
  fig.add_trace(
     go.Bar(name=mes, x=data2plot.index, y=data2plot.values),
     row=1, col= num + 1
  )
  df_media_mensal[meses[num]] = data=data2plot.values

fig.update_layout(height=600, 
                  width=1500, 
                  title_text="Média mensal por projeto",
                  )

fig.update_yaxes(range=[0,32])

fig.show()

### Tabela média mensal por equipe

In [ ]:
df_media_mensal.round(2)

,agosto,setembro,outubro,novembro,dezembro
PSE,9.99,10.14,8.88,2.26,2.58
Mídias,16.80,13.65,12.19,4.68,2.62
Parcerias,18.69,26.23,31.58,5.29,1.88
Eventos,11.96,12.94,12.24,7.09,4.42
PS,15.60,16.11,16.92,4.84,3.63


### Média geral por mês

In [ ]:
df_media_mensal.mean().round(2)

agosto      14.61
setembro    15.81
outubro     16.36
novembro     4.83
dezembro     3.03
dtype: float64

### Tempo membro por equipe a cada mês

In [ ]:
fig = make_subplots(rows=11)

for pos,equipe in enumerate(df_20_2['Equipe'].unique()):
  if equipe not in mask:
    plot = px.bar(df_20_2[df_20_2['Equipe'] == equipe],
                  x="user", 
                  y="Duração (s)", 
                  barmode="group",
                  facet_row="Equipe", 
                  facet_col="mes")
    #fig.add_trace(plot,pos,1)
    plot.update_layout(title={'text': equipe})
    plot.show()


# Sanity check 2021.1

In [ ]:
coh_21_1 = gc.open('2021.1 Controle de Horas Mensais - Inderios')

In [ ]:
nomes = coh_21_1.worksheets()

df_21_1 = pd.DataFrame()

#Cria dataframe unificado com todos os registros de todo mundo
for aba in nomes:
  if 'Auxiliar' in aba.title and aba.title != 'Base (Auxiliar)':
    df_21_1 = df_21_1.append(cria_df_2(aba), ignore_index=True)


-Tirar Artur (com registros do kelvin), Teodoro (somente 3 registros), Vitor e Anselmo (formato de data diferente)

In [ ]:
remover_nomes = [
                 'Artur',
                 'Teodoro Mascarenhas',
                 'Vitor Paiva',
                 'Anselmo'
              ]

mask = df_21_1['user'].isin(remover_nomes)
df_21_1 = df_21_1[~mask]

**Pegar somente entradas maiores que 10 segundos**

In [ ]:
df_21_1 = df_21_1[df_21_1['Duração (s)'] > 0.003]

Cria as colunas ano e mes

In [ ]:
df_21_1 = new_columns(df_21_1)

Somente dados de agosto a dezembro de 2020


In [ ]:
df_21_1 = filter_after(df_21_1,'mes',3)
df_21_1 = filter_before(df_21_1,'mes',7)
df_21_1 = filter_after(df_21_1,'ano',2021)

Corrige as atividades que possuem reunião no nome e possuem mais de 4 horas para ficarem somente com 2 horas.

In [ ]:
# Analisa quais outliers são reuniões
out = outliers(df_21_1, limite=4)

# Coloca os outliers que possuem reunião no nome com duracao de 2 horas
index_reunioes = out[list(map(lambda df_row: 'reuni' in df_row.lower(), out['Descrição da atividade']))].index

df_21_1.loc[index_reunioes, 'Duração (s)'] = 2
out.drop(index_reunioes, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Remove as entradas de quando esqueceram o toggl desligado: são entradas que possuem descrição, projeto e data de início iguais e uma linha com fim (de quando o toggl foi desligado) e uma linha com campo fim vazio (de quando o toggl estava ligado e registrou de madrugada)

In [ ]:
df_len = len(df_21_1)

# As entradas que não possuem fim estão duplicadas
indice_remover = out[(out.duplicated(['Descrição da atividade', 'Equipe','Início'], keep=False)) & (out['Fim'] == '')].index
df_21_1.drop(indice_remover, inplace=True)
out.drop(indice_remover,inplace=True)

print('{} linhas removidas'.format(df_len - len(df_21_1)))

0 linhas removidas


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Corrige atividades com duração maior que 4 horas

In [ ]:
# Atividades com duração maior que 4.1 horas ficam com duração média das atividades
media = df_21_1.loc[(df_21_1['Duração (s)'] <= 4) & (df_21_1['Duração (s)'] > 0.1)]['Duração (s)'].mean()

df_21_1.loc[out.index, 'Duração (s)'] = media

In [ ]:
df_21_1['Equipe'].unique()

array(['Diretoria Estratégica', 'Eventos', 'Parcerias', 'Outros',
       'Projeto extra', 'Pesquisa', 'Planejamento estratégico',
       'Salão Fisio e Forma', 'Estevão Andrade P.', 'Mídias', 'Geral',
       'DMKT', 'Medra', 'PE', 'Espaço de Évora', 'RH', 'Editora Voo',
       'Treinamentos', 'Processos Internos', 'Processo Seletivo',
       'Be Change', 'Frattezi Açaí e sorvetes', 'PS', 'ECP', 'outros',
       'DRH', 'DG', 'DP', 'GP', 'Reuniões', 'Fenômenos de Transporte',
       'Z - ECP - 2021.1', 'Z - Reuniões', 'Conselho',
       'Reuniões do Núcleo', 'EPR', 'Z - PS', 'Diretorias e Gerências',
       'Diretoria de Projetos', 'Slack', 'Clube do Livro', 'ESTÁGIO',
       'Be change', 'Extra', 'GRH', 'Diretoria de Marketing',
       'Diretoria de MKT', 'RG', 'Betânia Park', 'Mec Roq'], dtype=object)

# Piloto process mining Padoque

In [ ]:
from pm4py.objects.conversion.log import converter as log_converter 
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.visualization.petrinet import visualizer as pn_visualizer

/usr/local/lib/python3.7/dist-packages/pm4py/visualization/petrinet/__init__.py:20: UserWarning:

please use the pm4py.visualization.petri_net package instead



In [ ]:
def import_csv(file_path, separator=','):
    event_log = pd.read_csv(file_path, sep=separator)
    event_log = pm4py.format_dataframe(event_log, activity_key='description',case_id='project', timestamp_key='date', timest_format='%d/%m/%Y')
    start_activities = pm4py.get_start_activities(event_log)
    end_activities = pm4py.get_end_activities(event_log)
    print('Start activities: {}\nEnd activities: {}'.format(start_activities,end_activities))
    return event_log

def import_aba(file_path, name):
    df = pd.read_excel(path, sheet_name= name)
    return pm4py.format_dataframe(df,
                                  activity_key='description',
                                  case_id='project',
                                  timestamp_key='date',
                                  timest_format='%d/%m/%Y')

def getFilteredLog(log, filtered_value, column='case:concept:name'):
    return log[log[column] == filtered_value]

In [ ]:
events = pd.read_csv(r'/Dados_padoque_Padoque_original.csv',decimal=',')

start = [row.replace('T',' ')[:-6] for row in events.loc[:,'start']]
end = [row.replace('T',' ')[:-6] for row in events.loc[:,'end']]
updated = [row.replace('T',' ')[:-6] for row in events.loc[:,'updated']]

events.loc[:,'start'] = start
events.loc[:,'end'] = end
events.loc[:,'updated'] = updated

In [ ]:
# event_log = pm4py.format_dataframe(events, 
#                                    activity_key='task',
#                                    case_id='client',
#                                    timestamp_key='start', 
#                                    timest_format='%Y/%m/%d')

event_log = events.rename(columns=
                          {'start': 'time:timestamp', 
                           'user': 'case:concept:name', 
                           'task': 'concept:name', 
                           #'resource': 'org:resource'
                           })

## Convert to log format 
log = log_converter.apply(event_log)
net, initial_marking, final_marking = alpha_miner.apply(log)

gviz = pn_visualizer.apply(net, initial_marking, final_marking,
                           parameters=parameters,
                           variant=pn_visualizer.Variants.FREQUENCY,
                           log=log)
pn_visualizer.view(gviz)

NameError: ignored